In [5]:
import pyodbc
import pandas as pd
import datetime as dt
from pyspark.sql import SparkSession,Row
from pyspark.sql.functions import *
from pyspark.sql import functions as F
from pyspark.sql.types import *


import os 
import os.path as path
import datetime as dt
import sys
#criando um objeto sparksession object e um appName 
spark=SparkSession.builder.master("local[1]")\
        .appName("movie")\
        .config("spark.driver.extraClassPath","/Users/eduardoalberto/opt/spark/jars/mssql-jdbc-7.4.1.jre8.jar" ) \
        .getOrCreate()

sc = spark.sparkContext



server = 'localhost'
database = 'DBDWP511'
username = 'sa'
password = 'Numsey@Password!'
cnxn = pyodbc.connect('DRIVER={ODBC Driver 17 for SQL Server};SERVER='+server+';DATABASE='+database+';UID='+username+';PWD='+ password)
cursor = cnxn.cursor()

23/05/05 22:30:58 WARN Utils: Your hostname, Eduardos-MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 192.168.3.5 instead (on interface en0)
23/05/05 22:30:58 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/05/05 22:30:58 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/05/05 22:31:15 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors


OperationalError: ('HYT00', '[HYT00] [Microsoft][ODBC Driver 17 for SQL Server]Login timeout expired (0) (SQLDriverConnect)')

usando python puro

In [ ]:
tsql = "select * from DimDate;"
with cursor.execute(tsql):
    for row in cursor:
        print(str(row[0:]))
    

usando pandas para conectar ao banco

In [2]:
# tables = pd.read_sql_query("select * from INFORMATION_SCHEMA.TABLES;",cnxn)
DimEmployee = pd.read_sql_query("select * from DimEmployee;",cnxn)
DimDate = pd.read_sql_query("select * from DimDate;",cnxn)

DimEmployee["DateKey"] = pd.to_datetime(DimEmployee['StartDate']).dt.strftime('%Y%m%d')
DimEmployee["DateKey"] = pd.to_numeric(DimEmployee["DateKey"])


# t1 =DimEmployee[(DimEmployee["DateKey"] >= "20050101")]
t1 =DimEmployee[["DateKey","FirstName","LastName","Title","EmployeeKey","ParentEmployeeKey","EmployeeNationalIDAlternateKey","ParentEmployeeNationalIDAlternateKey","SalesTerritoryKey","SalariedFlag"]]
t2 =DimDate[["DateKey","DayNumberOfWeek","EnglishMonthName"]]

# display(t1.dtypes)
# display(t2.dtypes)

t3 = pd.merge(t1, t2, on="DateKey")
# t3 = t1.join(t2, lsuffix="_left", rsuffix="_right")
display(t3)
# t3.dtypes



,DateKey,FirstName,LastName,Title,EmployeeKey,ParentEmployeeKey,EmployeeNationalIDAlternateKey,ParentEmployeeNationalIDAlternateKey,SalesTerritoryKey,SalariedFlag,DayNumberOfWeek,EnglishMonthName
0,20060128,Guy,Gilbert,Production Technician - WC60,1,18.0,14417807,None,11,False,7,January
1,20060826,Kevin,Brown,Marketing Assistant,2,7.0,253022876,None,11,False,7,August
2,20070611,Roberto,Tamburello,Engineering Manager,3,14.0,509647174,None,11,True,2,June
3,20070705,Rob,Walters,Senior Tool Designer,4,3.0,112457891,None,11,False,5,July
4,20091228,Rob,Walters,Senior Tool Designer,5,3.0,112457891,None,11,False,2,December
...,...,...,...,...,...,...,...,...,...,...,...,...
291,20111229,Ranjit,Varkey Chudukatil,Sales Representative,292,290.0,134219713,None,7,True,5,December
292,20120430,Tete,Mensa-Annan,Sales Representative,293,272.0,90836195,None,1,True,2,April
293,20121012,Syed,Abbas,Pacific Sales Manager,294,277.0,481044938,None,11,True,6,October
294,20121228,Rachel,Valdez,Sales Representative,295,290.0,954276278,None,8,True,6,December


In [10]:
t2 = spark.read.jdbc("jdbc:sqlserver://localhost:1433;databaseName=DBDWP511", "DimCustomer",properties={"user": "sa", "password": "Numsey@Password!"})
t3 = spark.read.jdbc("jdbc:sqlserver://localhost:1433;databaseName=DBDWP511", "FactSurveyResponse",properties={"user": "sa", "password": "Numsey@Password!"})

display(t2.dtypes)
display(t3.dtypes)

[('CustomerKey', 'int'),
 ('GeographyKey', 'int'),
 ('CustomerAlternateKey', 'string'),
 ('Title', 'string'),
 ('FirstName', 'string'),
 ('MiddleName', 'string'),
 ('LastName', 'string'),
 ('NameStyle', 'boolean'),
 ('BirthDate', 'date'),
 ('MaritalStatus', 'string'),
 ('Suffix', 'string'),
 ('Gender', 'string'),
 ('EmailAddress', 'string'),
 ('YearlyIncome', 'decimal(19,4)'),
 ('TotalChildren', 'int'),
 ('NumberChildrenAtHome', 'int'),
 ('EnglishEducation', 'string'),
 ('SpanishEducation', 'string'),
 ('FrenchEducation', 'string'),
 ('EnglishOccupation', 'string'),
 ('SpanishOccupation', 'string'),
 ('FrenchOccupation', 'string'),
 ('HouseOwnerFlag', 'string'),
 ('NumberCarsOwned', 'int'),
 ('AddressLine1', 'string'),
 ('AddressLine2', 'string'),
 ('Phone', 'string'),
 ('DateFirstPurchase', 'date'),
 ('CommuteDistance', 'string')]

[('SurveyResponseKey', 'int'),
 ('DateKey', 'int'),
 ('CustomerKey', 'int'),
 ('ProductCategoryKey', 'int'),
 ('EnglishProductCategoryName', 'string'),
 ('ProductSubcategoryKey', 'int'),
 ('EnglishProductSubcategoryName', 'string'),
 ('Date', 'timestamp')]

In [ ]:
t2 = spark.read.jdbc("jdbc:sqlserver://localhost:1433;databaseName=DBDWP511", "DimCustomer",properties={"user": "sa", "password": "Numsey@Password!"})
t3 = spark.read.jdbc("jdbc:sqlserver://localhost:1433;databaseName=DBDWP511", "FactSurveyResponse",properties={"user": "sa", "password": "Numsey@Password!"})

display(t2.dtypes)
display(t3.dtypes)

[('CustomerKey', 'int'),
 ('GeographyKey', 'int'),
 ('CustomerAlternateKey', 'string'),
 ('Title', 'string'),
 ('FirstName', 'string'),
 ('MiddleName', 'string'),
 ('LastName', 'string'),
 ('NameStyle', 'boolean'),
 ('BirthDate', 'date'),
 ('MaritalStatus', 'string'),
 ('Suffix', 'string'),
 ('Gender', 'string'),
 ('EmailAddress', 'string'),
 ('YearlyIncome', 'decimal(19,4)'),
 ('TotalChildren', 'int'),
 ('NumberChildrenAtHome', 'int'),
 ('EnglishEducation', 'string'),
 ('SpanishEducation', 'string'),
 ('FrenchEducation', 'string'),
 ('EnglishOccupation', 'string'),
 ('SpanishOccupation', 'string'),
 ('FrenchOccupation', 'string'),
 ('HouseOwnerFlag', 'string'),
 ('NumberCarsOwned', 'int'),
 ('AddressLine1', 'string'),
 ('AddressLine2', 'string'),
 ('Phone', 'string'),
 ('DateFirstPurchase', 'date'),
 ('CommuteDistance', 'string')]

[('SurveyResponseKey', 'int'),
 ('DateKey', 'int'),
 ('CustomerKey', 'int'),
 ('ProductCategoryKey', 'int'),
 ('EnglishProductCategoryName', 'string'),
 ('ProductSubcategoryKey', 'int'),
 ('EnglishProductSubcategoryName', 'string'),
 ('Date', 'timestamp')]

In [ ]:
t2 = spark.read.jdbc("jdbc:sqlserver://localhost:1433;databaseName=DBDWP511", "DimCustomer",properties={"user": "sa", "password": "Numsey@Password!"})
t3 = spark.read.jdbc("jdbc:sqlserver://localhost:1433;databaseName=DBDWP511", "FactSurveyResponse",properties={"user": "sa", "password": "Numsey@Password!"})

display(t2.dtypes)
display(t3.dtypes)

[('CustomerKey', 'int'),
 ('GeographyKey', 'int'),
 ('CustomerAlternateKey', 'string'),
 ('Title', 'string'),
 ('FirstName', 'string'),
 ('MiddleName', 'string'),
 ('LastName', 'string'),
 ('NameStyle', 'boolean'),
 ('BirthDate', 'date'),
 ('MaritalStatus', 'string'),
 ('Suffix', 'string'),
 ('Gender', 'string'),
 ('EmailAddress', 'string'),
 ('YearlyIncome', 'decimal(19,4)'),
 ('TotalChildren', 'int'),
 ('NumberChildrenAtHome', 'int'),
 ('EnglishEducation', 'string'),
 ('SpanishEducation', 'string'),
 ('FrenchEducation', 'string'),
 ('EnglishOccupation', 'string'),
 ('SpanishOccupation', 'string'),
 ('FrenchOccupation', 'string'),
 ('HouseOwnerFlag', 'string'),
 ('NumberCarsOwned', 'int'),
 ('AddressLine1', 'string'),
 ('AddressLine2', 'string'),
 ('Phone', 'string'),
 ('DateFirstPurchase', 'date'),
 ('CommuteDistance', 'string')]

[('SurveyResponseKey', 'int'),
 ('DateKey', 'int'),
 ('CustomerKey', 'int'),
 ('ProductCategoryKey', 'int'),
 ('EnglishProductCategoryName', 'string'),
 ('ProductSubcategoryKey', 'int'),
 ('EnglishProductSubcategoryName', 'string'),
 ('Date', 'timestamp')]

In [224]:
t2.join(t3, t2.CustomerKey == t3.CustomerKey, "inner").select(t2.CustomerKey,t3.DateKey).show()
# t4.dtypes
# t4.select(
#      "GeographyKey"
#     ,"CustomerAlternateKey"
#     ,"Title"
#     ,"FirstName"
#     ,"MiddleName"
#     ,"LastName"
#     ,"NameStyle"
#     ,"BirthDate"
#     ,"MaritalStatus"
#     ,"Suffix"
#     ,"Gender"
#     ,"EmailAddress"
#     ,"YearlyIncome"
#     ,"TotalChildren"
#     ,"NumberChildrenAtHome"
#     ,"EnglishEducation"
#     ,"HouseOwnerFlag"
#     ,"NumberCarsOwned"
#     ,"AddressLine1"
#     ,"ProductCategoryKey"
#     ,"EnglishProductCategoryName"
#     ,"ProductSubcategoryKey"
#     ,"EnglishProductSubcategoryName"
#     ,"Date").show()

+-----------+--------+
|CustomerKey| DateKey|
+-----------+--------+
|      11033|20120310|
|      11033|20120310|
|      11033|20120310|
|      16339|20120329|
|      17753|20120318|
|      18944|20120325|
|      23271|20120324|
|      28759|20120427|
|      28759|20120427|
|      28836|20120229|
|      28836|20120229|
|      19204|20120328|
|      18800|20120318|
|      11141|20120417|
|      20497|20120301|
|      18800|20120318|
|      11141|20120417|
|      20497|20120301|
|      11316|20120325|
|      11316|20120325|
+-----------+--------+
only showing top 20 rows



In [223]:
from pyspark.sql.functions import json_tuple
jstr1 = u'{"id":12345,"foo":"bar","bodyid":111000,"name":"foobar","sub_jsonid":54321,"sub_sub_jsoncol1":20,"col2":"somethong"}'
jstr1 = u'{"id":12345,"foo":"bar","bodyid":111000,"name":"foobar","sub_jsonid":54321,"sub_sub_jsoncol1":20,"col2":"somethong"}'
jstr2 = u'{"id":12346,"foo":"baz","bodyid":111002,"name":"barfoo","sub_jsonid":23456,"sub_sub_jsoncol1":30,"col2":"something else"}'
jstr3 = u'{"id":43256,"foo":"foobaz","bodyid":20192,"name":"bazbar","sub_jsonid":39283,"sub_sub_jsoncol1":50,"col2":"another thing"}'
jsonString="""{"Zipcode":704,"ZipCodeType":"STANDARD","City":"PARC PARQUE","State":"PR"}"""

df01 = spark.createDataFrame([Row(json=jstr1),Row(json=jstr2),Row(json=jstr3)])
df=spark.createDataFrame([(1, jsonString)],["id","value"])

df01.show(truncate=False)
df.show(truncate=False)

df02 = df01.select(df01.json, json_tuple(df01.json,"id", "foo", "bodyid","name","foobar","sub_jsonid","sub_sub_jsoncol1","col2")) \
    .toDF("json", "id", "foo", "bodyid","name","foobar","sub_jsonid","sub_sub_jsoncol1","col2") \
    # .show(truncate=False)

df.select(col("id"),json_tuple(col("value"),"Zipcode","ZipCodeType","City","State")) \
.toDF("id","Zipcode","ZipCodeType","City","State") \
.show(truncate=False)



df02.createOrReplaceTempView("tempAnalise")

spark.sql("select id, foo, bodyid,name,foobar,sub_jsonid,sub_sub_jsoncol1,col2 from tempAnalise").show(truncate=False)







+--------------------------------------------------------------------------------------------------------------------------+
|json                                                                                                                      |
+--------------------------------------------------------------------------------------------------------------------------+
|{"id":12345,"foo":"bar","bodyid":111000,"name":"foobar","sub_jsonid":54321,"sub_sub_jsoncol1":20,"col2":"somethong"}      |
|{"id":12346,"foo":"baz","bodyid":111002,"name":"barfoo","sub_jsonid":23456,"sub_sub_jsoncol1":30,"col2":"something else"} |
|{"id":43256,"foo":"foobaz","bodyid":20192,"name":"bazbar","sub_jsonid":39283,"sub_sub_jsoncol1":50,"col2":"another thing"}|
+--------------------------------------------------------------------------------------------------------------------------+

+---+--------------------------------------------------------------------------+
|id |value                                 

In [178]:
base= [{
  "RecordNumber": 2,
  "Zipcode": 704,
  "ZipCodeType": "STANDARD",
  "City": "PASEO COSTA DEL SUR",
  "State": "PR"
},
{
  "RecordNumber": 10,
  "Zipcode": 709,
  "ZipCodeType": "STANDARD",
  "City": "BDA SAN LUIS",
  "State": "PR"
}]

df02 = spark.createDataFrame(base)
df02.show(truncate=False)


+-------------------+------------+-----+-----------+-------+
|City               |RecordNumber|State|ZipCodeType|Zipcode|
+-------------------+------------+-----+-----------+-------+
|PASEO COSTA DEL SUR|2           |PR   |STANDARD   |704    |
|BDA SAN LUIS       |10          |PR   |STANDARD   |709    |
+-------------------+------------+-----+-----------+-------+



In [201]:
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DoubleType, ArrayType
import json


jstr1 = u'{"header":{"id":12345,"foo":"bar"},"body":{"id":111000,"name":"foobar","sub_json":{"id":54321,"sub_sub_json":{"col1":20,"col2":"somethong"}}}}'
jstr2 = u'{"header":{"id":12346,"foo":"baz"},"body":{"id":111002,"name":"barfoo","sub_json":{"id":23456,"sub_sub_json":{"col1":30,"col2":"something else"}}}}'
jstr3 = u'{"header":{"id":43256,"foo":"foobaz"},"body":{"id":20192,"name":"bazbar","sub_json":{"id":39283,"sub_sub_json":{"col1":50,"col2":"another thing"}}}}'
# schema = StructType([StructField('json', StringType(), True)])
# cria Dataframe
df0 = spark.createDataFrame([Row(json=jstr1),Row(json=jstr2),Row(json=jstr3)],schema)
# new_df = spark.read.json(df.rdd.map(lambda r: r.json))
# new_df.printSchema()

data = json.loads(jstr1)
print(data.get("body"))



{'id': 111000, 'name': 'foobar', 'sub_json': {'id': 54321, 'sub_sub_json': {'col1': 20, 'col2': 'somethong'}}}


In [214]:
dummyDB01 = [
   {
      "userId":1,
      "id":1,
      "title":"sunt aut facere repellat provident occaecati excepturi optio reprehenderit",
      "body":"quia et suscipit\nsuscipit recusandae consequuntur expedita et cum\nreprehenderit molestiae ut ut quas totam\nnostrum rerum est autem sunt rem eveniet architecto"
   },
   {
      "userId":1,
      "id":2,
      "title":"qui est esse",
      "body":"est rerum tempore vitae\nsequi sint nihil reprehenderit dolor beatae ea dolores neque\nfugiat blanditiis voluptate porro vel nihil molestiae ut reiciendis\nqui aperiam non debitis possimus qui neque nisi nulla"
   },
   {
      "userId":1,
      "id":3,
      "title":"ea molestias quasi exercitationem repellat qui ipsa sit aut",
      "body":"et iusto sed quo iure\nvoluptatem occaecati omnis eligendi aut ad\nvoluptatem doloribus vel accusantium quis pariatur\nmolestiae porro eius odio et labore et velit aut"
   },
   {
      "userId":1,
      "id":4,
      "title":"eum et est occaecati",
      "body":"ullam et saepe reiciendis voluptatem adipisci\nsit amet autem assumenda provident rerum culpa\nquis hic commodi nesciunt rem tenetur doloremque ipsam iure\nquis sunt voluptatem rerum illo velit"
   }
   
   
]

df002 = spark.createDataFrame(dummyDB01)
df002.show(truncate=False)


+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+---+--------------------------------------------------------------------------+------+
|body                                                                                                                                                                                                             |id |title                                                                     |userId|
+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+---+--------------------------------------------------------------------------+------+
|quia et suscipit\nsuscipit recusandae consequuntur expedita et cum\nreprehenderit molestiae ut ut quas to

In [222]:
dummyDB02 = [
   {
      "id":1,
      "name":"Leanne Graham",
      "username":"Bret",
      "email":"Sincere@april.biz",
      "address":{
         "street":"Kulas Light",
         "suite":"Apt. 556",
         "city":"Gwenborough",
         "zipcode":"92998-3874",
         "geo":{
            "lat":"-37.3159",
            "lng":"81.1496"
         }
      },
      "phone":"1-770-736-8031 x56442",
      "website":"hildegard.org",
      "company":{
         "name":"Romaguera-Crona",
         "catchPhrase":"Multi-layered client-server neural-net",
         "bs":"harness real-time e-markets"
      }
   },
   {
      "id":2,
      "name":"Ervin Howell",
      "username":"Antonette",
      "email":"Shanna@melissa.tv",
      "address":{
         "street":"Victor Plains",
         "suite":"Suite 879",
         "city":"Wisokyburgh",
         "zipcode":"90566-7771",
         "geo":{
            "lat":"-43.9509",
            "lng":"-34.4618"
         }
      },
      "phone":"010-692-6593 x09125",
      "website":"anastasia.net",
      "company":{
         "name":"Deckow-Crist",
         "catchPhrase":"Proactive didactic contingency",
         "bs":"synergize scalable supply-chains"
      }
   }
   
]
df003 = spark.createDataFrame(dummyDB02)

df003 = df003.select(df003.email).show(truncate=False)

+-----------------+
|email            |
+-----------------+
|Sincere@april.biz|
|Shanna@melissa.tv|
+-----------------+



In [ ]:
+------------------------------------------------------------------------------------------------------------------------------+-------------------------------------------------------------------------------------------------------------------+-----------------+---+-------------+---------------------+---------+-------------+
|address                                                                                                                       |company                                                                                                            |email            |id |name         |phone                |username |website      |
+------------------------------------------------------------------------------------------------------------------------------+-------------------------------------------------------------------------------------------------------------------+-----------------+---+-------------+---------------------+---------+-------------+
|{geo -> {lng=81.1496, lat=-37.3159}, zipcode -> 92998-3874, suite -> Apt. 556, city -> Gwenborough, street -> Kulas Light}    |{name -> Romaguera-Crona, bs -> harness real-time e-markets, catchPhrase -> Multi-layered client-server neural-net}|Sincere@april.biz|1  |Leanne Graham|1-770-736-8031 x56442|Bret     |hildegard.org|
|{geo -> {lng=-34.4618, lat=-43.9509}, zipcode -> 90566-7771, suite -> Suite 879, city -> Wisokyburgh, street -> Victor Plains}|{name -> Deckow-Crist, bs -> synergize scalable supply-chains, catchPhrase -> Proactive didactic contingency}      |Shanna@melissa.tv|2  |Ervin Howell |010-692-6593 x09125  |Antonette|anastasia.net|

### estudo sql

In [42]:
t2 =  t2.withColumn("YEAR",(substring("BirthDate",1,4)))
t2 =  t2.withColumn("RANK", when((col("YEAR") >= "1982") & (col("YEAR") <= "1990") ,lit(1)).otherwise(0)) #.select(col("RANK")).show()
# df.withColumn(my_column, when((col(my_column) < '1900-01-01') | (col(my_column) > '2019-12-09 17:01:37.774418'), lit(None)).otherwise(col(my_column)))
# t2.printSchema()
t2.select(t2.YEAR, t2.RANK).show()


+----+----+
|YEAR|RANK|
+----+----+
|1971|   0|
|1976|   0|
|1971|   0|
|1973|   0|
|1979|   0|
|1976|   0|
|1976|   0|
|1969|   0|
|1975|   0|
|1969|   0|
|1969|   0|
|1969|   0|
|1979|   0|
|1979|   0|
|1973|   0|
|1984|   1|
|1984|   1|
|1949|   0|
|1955|   0|
|1983|   1|
+----+----+
only showing top 20 rows



In [4]:
t2 = spark.read.jdbc("jdbc:sqlserver://localhost:1433;databaseName=DBDWP511", "DimCustomer",properties={"user": "sa", "password": "Numsey@Password!"})
t3 = spark.read.jdbc("jdbc:sqlserver://localhost:1433;databaseName=DBDWP511", "FactSurveyResponse",properties={"user": "sa", "password": "Numsey@Password!"})

In [1]:
class Person:
  def __init__(self, name, age):
    self.name = name
    self.age = age

p1 = Person("John", 36)

print(p1.name)
print(p1.age)

John
36


### Uso de classe 

In [23]:
pfile = '/Users/eduardoalberto/LoadFile/BR-Football-Dataset.csv'
nfile = '/Users/eduardoalberto/LoadFile/BR-Football-DatasetOld.csv'

class carga:
    def __init__(self,spark):
        self.__spark = spark
        if path.isfile(pfile):
            df = (self.__spark.read
                        .format("csv")
                        .option("delimiter", ",")
                        .option("header", True)
                        .option("inferSchema", True)
                        .load(pfile))
        else:
                df = (self.__spark.read
                    .format("csv")
                    .option("delimiter", ",")
                    .option("header", True)
                    .option("inferSchema", True)
                    .load(nfile))
        return df.where(col("tournament")=="Copa do Brasil").show()
carga(spark)

            


+--------------+-------------------+---------+---------+--------------+-----------+-----------+-----------+-----------+----------+----------+-------------------+----------+-------+-------+---------+---------+-------------+
|    tournament|               home|home_goal|away_goal|          away|home_corner|away_corner|home_attack|away_attack|home_shots|away_shots|               time|      date|ht_diff|at_diff|ht_result|at_result|total_corners|
+--------------+-------------------+---------+---------+--------------+-----------+-----------+-----------+-----------+----------+----------+-------------------+----------+-------+-------+---------+---------+-------------+
|Copa do Brasil|   Vasco Da Gama RJ|      0.0|      0.0|           ABC|        6.0|        2.0|      133.0|       82.0|       9.0|       6.0|2023-05-06 00:30:00|2023-03-17|    0.0|    0.0|     DRAW|     DRAW|          8.0|
|Copa do Brasil|             Gremio|      3.0|      0.0|   Ferroviario|       10.0|        3.0|       98.0| 